In [2]:
import math
import numpy as np

from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Reshape, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint

from gpaulo.game import suggest
from gpaulo.train import load_data

/home/gpaulo00/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# load data
train_x, train_y, test_x, test_y = load_data()

In [3]:
# load neural network
nn = load_model("model.h5")

In [4]:
# build neural network
nn = Sequential()
nn.add(Dense(9, input_dim=9, name="main_input"))
nn.add(Dense(27, activation="tanh", name="hidden1"))
nn.add(Dense(81, activation="tanh", name="hidden2"))
nn.add(Dropout(.1, name="single_dropout"))
nn.add(Dense(27, activation="tanh", name="hidden3"))
nn.add(Dense(9, activation="sigmoid", name="main_output"))

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
nn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (Dense)           (None, 9)                 90        
_________________________________________________________________
hidden1 (Dense)              (None, 27)                270       
_________________________________________________________________
hidden2 (Dense)              (None, 81)                2268      
_________________________________________________________________
single_dropout (Dropout)     (None, 81)                0         
_________________________________________________________________
hidden3 (Dense)              (None, 27)                2214      
_________________________________________________________________
main_output (Dense)          (None, 9)                 252       
Total params: 5,094
Trainable params: 5,094
Non-trainable params: 0
_________________________________________________________________


In [4]:
# checkpoint and early stop
callbacks = [
    ModelCheckpoint("weights.best.hdf5", monitor="val_acc", verbose=1, save_best_only=True, mode="max"),
    EarlyStopping(monitor="val_acc", patience=50, mode="max"),
]

# train network
nn.fit(train_x, train_y, epochs=500, batch_size=32, validation_data=(test_x, test_y), callbacks=callbacks)

# evaluate the network
scores = nn.evaluate(train_x, train_y)
print("\n%s: %.2f%%" % (nn.metrics_names[1], scores[1]*100))

Train on 30483 samples, validate on 72 samples
Epoch 1/500
30483/30483 [==============================] - 9s 295us/step - loss: 0.0585 - acc: 0.9780 - val_loss: 0.0070 - val_acc: 0.9969

Epoch 00001: val_acc improved from -inf to 0.99691, saving model to weights.best.hdf5
Epoch 2/500
30483/30483 [==============================] - 8s 257us/step - loss: 0.0577 - acc: 0.9785 - val_loss: 0.0067 - val_acc: 0.9969

Epoch 00002: val_acc did not improve from 0.99691
Epoch 3/500
30483/30483 [==============================] - 8s 257us/step - loss: 0.0584 - acc: 0.9782 - val_loss: 0.0067 - val_acc: 0.9969

Epoch 00003: val_acc did not improve from 0.99691
Epoch 4/500
30483/30483 [==============================] - 7s 238us/step - loss: 0.0575 - acc: 0.9788 - val_loss: 0.0067 - val_acc: 0.9969

Epoch 00004: val_acc did not improve from 0.99691
Epoch 5/500
30483/30483 [==============================] - 6s 183us/step - loss: 0.0576 - acc: 0.9785 - val_loss: 0.0066 - val_acc: 0.9969

Epoch 00005: val_

In [10]:
nn.save("model.h5")